# Information
$Writer: Junhyuck\ Woo$ <br>
$Std.ID: 20145337$<br>
$Assignment09:\ Multi-label\ classification\ using\ neural\ networks$<br>
$Deadline: May\ 28, 2020$
<hr>

# Library

In [0]:
import matplotlib.pyplot as plt; import numpy as np

# Data

In [0]:
file_data   = "/content/drive/My Drive/Spring|2020/Machine_Learning/CAU-MachineLearning/Assignment09/mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

# Split data as train one and test one
Train: [1~6000] <br>
Test: [6001 ~ 10000]

In [0]:
train_m = 6000
test_m = 4000
train_image = list_image[:,:6000]
test_image = list_image[:, 6000:]

# Neural Network Architecture
```mermaid
graph LR
    A([input layer : x])-->B([first hidden layer : y])-->C([second hidden layer : z])-->D([output layer : h])
```

```mermaid
graph LR
    A([x])-- fully connected : u -->B([y_])-- sigmoid -->C([y])-- fully connected : v -->D([z_])-- sigmoid -->E([z])-- fully connected : w -->F([h_])-- sigmoid -->G([h])
```
* input layer : 784 (+ a bias)
* first hidden layer : 196 (+ a bias)
* second hidden layer : 49 (+ a bias)
* output layer : 10

In [0]:
mean = 0
sigma = 1
node_num = [784, 196, 49, 10]

network = []
for i, v in enumerate(node_num[:-1]):
    network.append(np.random.normal(mean, sigma, (node_num[i+1], node_num[i]+1)))

# Sigmoid Function (= activation function)
$sigma(z) = \frac{1}{1 + \exp(-z)}$

In [0]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

# Objective Function
$J(\theta) = \frac{1}{m} \sum_{i=1}^m \sum_{k=0}^{9}( - l^{(i)}_k \log( h^{(i)}_k ) - (1 - l^{(i)}_k) \log(1 - h^{(i)}_k) )$<br>
$\theta = (u, v, w)$

In [0]:
def one_hot(l):
    new_label = np.zeros(10)
    new_label[l] = 1
    return new_label
    

def j_func(h, index, m):
    real_label = one_hot(list_label(index))
    tmp = -real_label*np.log(h) - (1 - real_label)*np.log(1-h)
    return tmp        

# Gradient Descent (Back-propagation)

In [0]:
def optimize(model):
    alpha = 0.0002

# Optimization

In [78]:
# Varibales
count = 1
accuracy = [0, 0]
accuracy_history = [[], []]
j = [0, 0]
j_old = [0, 0]
j_history = [[], []]


while (True):
    # Reset the variables
    accuracy = [0, 0]
    j_old = j
    j = [0, 0]
    # Train
    for i, v in enumerate(train_image.T):
        # Attach Bias
        v = np.concatenate((np.ones(1), v))
        y_ = network[0].dot(v)
        y = sigmoid(y_)
        # Attach Bias
        y = np.concatenate((np.ones(1), y))
        z_ = network[1].dot(y)
        z = sigmoid(z_)
        # Attach Bias
        z = np.concatenate((np.ones(1), z))
        h_ = network[2].dot(z)
        h = sigmoid(h_)

        # Loss
        j[0] = j[0] + j_func(h, i, train_m)

        # Accuracy
        if (np.argmax(h) == list_label[i]):
            accuracy[0] = accuracy[0] + 1
      
    # Record the history
    accuracy_history[0].append(accuracy[0]/60)
    j_history[0].append(j[0]/train_m)


    # Test
    for i, v in enumerate(test_image.T):
        # Attach Bias
        v = np.concatenate((np.ones(1), v))
        y_ = network[0].dot(v)
        y = sigmoid(y_)
        # Attach Bias
        y = np.concatenate((np.ones(1), y))
        z_ = network[1].dot(y)
        z = sigmoid(z_)
        # Attach Bias
        z = np.concatenate((np.ones(1), z))
        h_ = network[2].dot(z)
        h = sigmoid(h_)

        # Loss
        j[1] = j[1] + j_func(h, i, test_m)

        # Accuracy
        if (np.argmax(h) == list_label[i]):
            accuracy[1] = accuracy[1] + 1
    
    # Record the history
    accuracy_history[1].append(accuracy[1]/60)
    j_history[1].append(j[1]/test_m)


    # Back-propagation





    # Break Point
    if (count==1000):
        break

    # Count Iteration
    count = count + 1

[0.81496452 0.00740866 0.02626402 0.30457664 0.96945818 0.35113964
 0.19153417 0.03814919 0.04822231 0.9998018 ]
7


# Check the history for display example

In [0]:
correct_examples = []
correct_examples_num = 0
misclassified_examples = []
misclassified_examples_num = 0

for i, v in enumerate(test_label):
    logic = (v == 

    if correct_examples_num <= 10:

    else:
       if misclassified_examples_num == 10:
           break
           
    if misclassified_examples_num < 10:
        
    else:
        if correct_examples_num == 10:
            break

___
#Result

# 1. Loss Curve

In [0]:
plt.figure(figsize=(15,9))
plt.plot(j_history[0], color='blue', label='Train')
plt.plot(j_history[1], color='red', label='Test')
plt.grid()
plt.legend()
plt.title('Training Accuracy', fontsize=40)
plt.xlabel('Iteration', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)

# 2. Accuravy Curve

In [0]:
plt.figure(figsize=(15,9))
plt.plot(accuracy_history[0], color='blue', label='Train Accuracy (%)')
plt.plot(accuracy_history[1], color='red', label='Test Accuracy (%)')
plt.grid()
plt.legend()
plt.title('Accuracy', fontsize=40)
plt.xlabel('Iteration', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)

# 3. Accuracy Value

# 4. Classification Example